### Deploy Hugging Face model to do sentiment analysis

We're using https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment

In [ ]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'cardiffnlp/twitter-xlm-roberta-base-sentiment',
	'HF_TASK':'text-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.26.0',
	pytorch_version='1.13.1',
	py_version='py39',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge', # ec2 instance type
	endpoint_name='roberta-base-sentiment' # to have a meaningfully named endpoint, but make sure not to use exact name of the model
)

predictor.predict({
	"inputs": "This is my cat, isn't it adorable",
})